## Data Ingestion stage 

In [20]:
import os 

In [21]:
%pwd 

'/Users/jihaad/Documents/2023_Learning/Project/CnnClassifier'

In [3]:
os.chdir("../")

In [4]:
%pwd 

'/Users/jihaad/Documents/2023_Learning/Project/CnnClassifier'

### entity

In [22]:
from collections import namedtuple
import os 

In [23]:
DataIngestionConfig = namedtuple("DataIngestionConfig",[
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [24]:
DataIngestionConfig

__main__.DataIngestionConfig

- define the return type for class or method

In [18]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path 
    source_URL:str 
    local_data_file:Path    
    unzip_dir:Path 

### Constant

- CONFIG_FILE_PATH 
- PARAMS_FILE_PATH 

### Configuration manager

In [25]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories

- extract file in config.yaml with return type is on entity

In [26]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

### Component

In [27]:
import os 
import urllib.request as request
from zipfile import ZipFile
from cnnClassifier import logger
from cnnClassifier.utils import get_size

we find some issue im zip file 
- after unzip the file we find 3 file and we only use 1 file name Petimages folder 
- so we will set to only use Petimages folder 
- afert we open the Petimage file 
- we find zero kb file and not image file
- so we will use only file with .jpg 

In [31]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f )]
    
    def _preprocess(self, zf: ZipFile, f:str, working_dir:str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
            
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)
    
    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            update_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in update_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)

- underscore before the name its mean hidden function

### Pipeline

In [33]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2023-09-27 23:22:30,125: INFO: common: yaml file: config/config.yaml loaded succesfully]
[2023-09-27 23:22:30,132: INFO: common: yaml file: params.yaml loaded succesfully]
[2023-09-27 23:22:30,146: INFO: common: created directory at: artifacts]
[2023-09-27 23:22:30,149: INFO: common: created directory at: artifacts/data_ingestion]


[2023-09-27 23:22:42,973: INFO: 3305050350: artifacts/data_ingestion/data.zip download! with following info 
Connection: close
Content-Length: 22329406
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "e33c4dcc8f22340234fac4b9da4a20b304912be521f658008139809163a077ea"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BD4C:1357D1:15F6B:1A18E:651454BC
Accept-Ranges: bytes
Date: Wed, 27 Sep 2023 16:13:50 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg1271-QPG
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1695831229.585530,VS0,VE1839
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: d433db310eccbf5b6a95bb73dea3a176ee681cea
Expires: Wed, 27 Sep 2023 16:18:50 GMT
Source-Age: 2

]
